I'm a beginner, so please be gentle with me.

In [ ]:
import os
# os.chdir('INPUT YOUR LOCAL DIR')

# DataSet & Library Loading

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import time, gc

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.autograd import Variable
from torch.optim import lr_scheduler

In [ ]:
class CFG:
    min_batch_size = 10_000  # max_batch_sizeの1000分の1程度が限界か
    max_batch_size = 100_000  # 大きいほどVRAMを食う
    batch_increase_rate = 2  # batch_sizeを変動させるときの係数
    epochs = 10_000  # 最大のepoch数
    folds = 10  # クロスバリデーションにおけるfold数
    seed = 42  # TODO 機能させる
    target = 'target'  # 目標変数
    model_path = "models"  # モデルを保存するフォルダ
    test_pred = []  # 各foldで作ったモデル別の、testを予測した結果のカラム名
    pred = 'pred'  # trainを予測した結果のカラム名
    increase_batch = 9999  # 何epoch間lossが改善しなかったbatchを増やすか
    early_stopping = 100  # 何epochでearly stoppingをするか
    lr = 1  # optimizerの学習率
    # min_lr = 1e-3  # 学習率の最低値
    lr_patience = 10  # 学習率を引き下げる間隔
    lr_factor = 0.5  # lrを何倍にするか
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    delta = 1e-4  # AUC等がいくつ変わらなかったら学習などをやめるか

In [ ]:
os.makedirs(CFG.model_path, exist_ok=True)

## FE

In [ ]:
# データセット読み込み
df_train = pd.read_csv("../input/tabular-playground-series-may-2022/train.csv")
df_test = pd.read_csv("../input/tabular-playground-series-may-2022/test.csv")
df_sub = pd.read_csv("../input/tabular-playground-series-may-2022/sample_submission.csv")
display(df_train.head())
display(df_test.head())
display(df_sub.head())

In [ ]:
display(df_train.shape)
display(df_test.shape)

In [ ]:
from matplotlib.colors import ListedColormap

In [ ]:
# https://www.kaggle.com/code/ambrosm/tpsmay22-advanced-keras
plt.rcParams['axes.facecolor'] = 'k'
plt.figure(figsize=(11, 5))
cmap = ListedColormap(["#ffd700", "#0057b8"])

ax = plt.subplot(1, 3, 1)
ax.scatter(df_train['f_02'], df_train['f_21'], s=1,
           c=df_train.target, cmap=cmap)
ax.set_xlabel('f_02')
ax.set_ylabel('f_21')
ax.set_aspect('equal')
ax0 = ax

ax = plt.subplot(1, 3, 2, sharex=ax0, sharey=ax0)
ax.scatter(df_train['f_05'], df_train['f_22'], s=1,
           c=df_train.target, cmap=cmap)
ax.set_xlabel('f_05')
ax.set_ylabel('f_22')
ax.set_aspect('equal')

ax = plt.subplot(1, 3, 3, sharex=ax0, sharey=ax0)
ax.scatter(df_train['f_00'] + df_train['f_01'], df_train['f_26'], s=1,
           c=df_train.target, cmap=cmap)
ax.set_xlabel('f_00 + f_01')
ax.set_ylabel('f_26')
ax.set_aspect('equal')

plt.tight_layout(w_pad=1.0)
plt.show()
plt.rcParams['axes.facecolor'] = '#0057b8' # blue

In [ ]:
df_all = pd.concat([df_train, df_test])

In [ ]:
df_all['i_02_21'] = (df_all.f_21 + df_all.f_02 > 5.2).astype(int) - (df_all.f_21 + df_all.f_02 < -5.3).astype(int)
df_all['i_05_22'] = (df_all.f_22 + df_all.f_05 > 5.1).astype(int) - (df_all.f_22 + df_all.f_05 < -5.4).astype(int)
i_00_01_26 = df_all.f_00 + df_all.f_01 + df_all.f_26
df_all['i_00_01_26'] = (i_00_01_26 > 5.0).astype(int) - (i_00_01_26 < -5.0).astype(int)

In [ ]:
# 不要データを削除
df_all.drop(['id'], axis=1, inplace=True)

In [ ]:
# NA埋め
df_all.fillna(df_all.mean(numeric_only=True), inplace=True)

In [ ]:
# f_27を分解
def splitter(text):
    arr = tuple(text)
    return arr

column_list = ['f_27_' + str(i) for i in range(10)]
df_all[column_list] = df_all.apply(lambda x: splitter(x['f_27']), axis=1, result_type='expand')

In [ ]:
# 新設したf_27を分解したものを、one-hot-encoding
# メモリがやばいのでやめる
# new_df = pd.get_dummies(df_all[column_list])
# df_all = pd.concat([df_all, new_df], axis=1)

In [ ]:
# f_27_0~9を数値に
for column in column_list:
    df_all[column] = df_all[column].apply(lambda x: ord(x)-ord('A'))

In [ ]:
# f_27_0~9の各種統計量を求める

df_all['f_27_mean'] =  df_all[column_list].mean(axis=1)
df_all['f_27_std'] =  df_all[column_list].std(axis=1)
df_all['f_27_sm'] = df_all['f_27_std'] / (df_all['f_27_mean']+0.001)


In [ ]:
from sklearn.linear_model import LinearRegression
lrm = LinearRegression()

In [ ]:
from tqdm import tqdm

In [ ]:
# とても遅い
def c2i(c):
    return ord(c)-ord('A')

def calc_linear_regression(txt):
    arr = [c2i(c) for c in txt]
    y = np.array(arr).reshape(-1, 1)
    x = np.arange(len(y)).reshape(-1, 1)
    lrm.fit(x, y)
    return pd.Series([lrm.coef_[0].item(), lrm.intercept_.item()])
tqdm.pandas()
df_all[['lr_a', 'lr_b']] = df_all['f_27'].progress_apply(calc_linear_regression)

In [ ]:
# f_27に出てくる文字が、何種類あるかを返す
def calc_cnt_of(txt):
    from collections import Counter
    cnt_of = Counter(list(txt))
    return len(cnt_of)

df_all['len_cnt_of'] = df_all['f_27'].apply(calc_cnt_of)

In [ ]:
# f_27に各文字が何回出てくるかを数える
def calc_char_cnt(txt):
    from collections import Counter
    import string
    cnt_of = Counter(list(txt))
    ret = []
    for c in string.ascii_uppercase:
        ret.append(cnt_of[c])
    return tuple(ret)

import string
column_list = []
for c in string.ascii_uppercase:
    column_list.append('f_27_cnt_' + c)

df_all[column_list] = df_all.apply(lambda x: calc_char_cnt(x['f_27']), axis=1, result_type='expand')

In [ ]:
# 不要になったカテゴリ変数を削除
df_all.drop(['f_27'], axis=1, inplace=True)

In [ ]:
# この時点で数値しかない。正規化
scaler = StandardScaler()
for column in df_all.columns:
    if column == CFG.target:
        continue
    if df_all[column].dtype == 'object':
        continue
    df_all[column] = scaler.fit_transform(df_all[column].values.reshape(-1, 1))

In [ ]:
df_train = df_all[:len(df_train)]
df_test = df_all[len(df_train):]

In [ ]:
df_train.to_pickle('train_fe.pkl')
df_test.to_pickle('test_fe.pkl')

In [ ]:
del df_all

# Pytorch

In [ ]:
df_train = pd.read_pickle('train_fe.pkl')
df_test = pd.read_pickle('test_fe.pkl')

## Define Pytorch Model

In [ ]:
class Net1(nn.Module):
    def __init__(self, input_size, output_size):
        # 7層
        super(Net1, self).__init__()
        self.first_bn = nn.BatchNorm1d(input_size, momentum=0.01)  # とりあえず入れてみた。momentumが小さいといいことあるんかな・・？
        self.fc1 = nn.Linear(input_size, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512, 384)
        self.bn2 = nn.BatchNorm1d(384)
        self.fc3 = nn.Linear(384, 256)
        self.bn3 = nn.BatchNorm1d(256)
        self.fc4 = nn.Linear(256, 128)
        self.bn4 = nn.BatchNorm1d(128)
        self.fc5 = nn.Linear(128, 64)
        self.bn5 = nn.BatchNorm1d(64)
        self.fc6 = nn.Linear(64, 32)
        self.bn6 = nn.BatchNorm1d(32)
        self.fc7 = nn.Linear(32, output_size)
        self.bn7 = nn.BatchNorm1d(output_size)

    def forward(self, x):
        # dropoutの後にbnを置いてはならない
        # nbは、活性化関数の前に置く
        x = self.first_bn(x)
        x = F.silu(self.bn1((self.fc1(x))))
        x = F.silu(self.bn2((self.fc2(x))))
        x = F.silu(self.bn3((self.fc3(x))))
        x = F.silu(self.bn4((self.fc4(x))))
        x = F.silu(self.bn5((self.fc5(x))))
        x = F.silu(self.bn6((self.fc6(x))))
        x = self.bn7((self.fc7(x)))
        return x

# Pytorch Loss Function

In [ ]:
criterion = nn.BCEWithLogitsLoss()

# Pytorch Training

In [ ]:
class LossChecker:
    # 指定した回数、loss等が改善されていなければTrueを返す
    def __init__(self, patience=20, strategy="min", delta=1e-6):
        self.patience = patience
        self.bef_epoch = 0
        self.strategy = strategy
        self.delta = delta
        if strategy == "max":
            self.val = -float('inf')
        else:
            self.val = float('inf')

    def step(self, epoch, loss):
        if self.strategy == "max":
            if self.val < loss:
                self.val = loss + self.delta
                self.bef_epoch = epoch
            if epoch - self.bef_epoch > self.patience:
                self.bef_epoch = epoch
                return True
            else:
                return False
        else:
            if self.val > loss:
                self.val = loss - self.delta
                self.bef_epoch = epoch
            if epoch - self.bef_epoch > self.patience:
                self.bef_epoch = epoch
                return True
            else:
                return False


In [ ]:
# 学習の対象とする特徴量を列挙する
all_features = df_train.columns.tolist()
all_features.remove(CFG.target)

In [ ]:
class CalcBatchSize:
    # バッチサイズを
    # 極大->ちょっと減少->最小->ちょっと増加->ちょっと増加->...の順で遷移させる
    # あんまり意味なさそう・・・というか悪影響しかなさそう
    def __init__(self, min_batch_size, max_batch_size, rate=1.2):
        self.min_batch_size = min_batch_size
        self.max_batch_size = max_batch_size
        self.current_batch_size = max_batch_size
        self.cnt = -1
        self.rate = rate
        self.state = 0  # 0->初回、 1->増加、 -1->現象

    def step(self):
        if self.state == 0:
            self.state = -1
            return self.current_batch_size
        elif self.state == -1:
            # minに到達するまで、減少させる
            self.current_batch_size = int(self.current_batch_size/self.rate)
            self.current_batch_size = max(self.current_batch_size, self.min_batch_size)
        else:
            # 増加させ続ける
            self.current_batch_size = int(self.current_batch_size*self.rate)+1
            self.current_batch_size = min(self.current_batch_size, self.max_batch_size)
        if self.current_batch_size == self.min_batch_size:
            self.state = 1
        return self.current_batch_size
            

In [ ]:
def fit(Net, n_splits, df_train, df_test, model_name_prefix):
    # TODO 引数を調整する

    df_train = df_train.sample(frac=1).reset_index(drop=True)  # df_trainをシャッフル

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=CFG.seed)

    for fold, (train_index, valid_index) in enumerate(skf.split(df_train, df_train[CFG.target])):
        print('-----', fold, '-----')
        # fold毎に初期化する設定
        # model, optimizer, scheduler, その他変数
        model = Net(len(all_features), 1).to(CFG.device)
        print(model)

        optimizer = torch.optim.Adam(model.parameters(), lr=CFG.lr)
        # scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'max', verbose=True, factor=CFG.lr_factor, min_lr=CFG.min_lr, patience=CFG.lr_patience)
        calc_batch_size = CalcBatchSize(CFG.min_batch_size, CFG.max_batch_size, CFG.batch_increase_rate)
        batch_size = calc_batch_size.step()
        early_stopping = LossChecker(CFG.early_stopping, "max", CFG.delta)
        increase_batch = LossChecker(CFG.increase_batch, "max", CFG.delta)

        train_loss_min = np.Inf
        auc_max = -np.Inf

        # データを分割する。fold毎に一回だけやる
        X_train = df_train[all_features].iloc[train_index]
        y_train = df_train.iloc[train_index][CFG.target]
        X_valid = df_train[all_features].iloc[valid_index]
        y_valid = df_train.iloc[valid_index][CFG.target]
        X_test = df_test[all_features]
        st_time = time.time()

        for epoch in range(CFG.epochs):
            train_loss_list = []

            # TODO Variableを使うのは古いらしい
            # TODO 元データがメモリに収まらず、pklで管理している場合でも動くようにする
            # ここら辺見て実装する
            # https://tips-memo.com/python-dataset-memory
            # https://qiita.com/mathlive/items/2a512831878b8018db02
            # https://tma15.github.io/blog/2020/03/08/pytorch-dataset%E3%81%AE%E8%AA%AD%E3%81%BF%E8%BE%BC%E3%81%BF%E3%81%AB%E3%81%8B%E3%81%8B%E3%82%8B%E3%83%A1%E3%83%A2%E3%83%AA%E6%B6%88%E8%B2%BB%E9%87%8F%E3%82%92%E7%AF%80%E7%B4%84%E3%81%99%E3%82%8B/
            x0_var = Variable(torch.FloatTensor(X_train.values))   # GPUで送るのはこっちのほうが早いが、VRAMの浪費になると思う
            y0_var = Variable(torch.FloatTensor(y_train.values))
            x_valid_var = Variable(torch.FloatTensor(X_valid.values)).to(CFG.device)  # valid_dataはちっちゃいので、最初に送っちゃう
            i = -1
            while True:
                i += 1
                # ミニバッチ学習
                start = i * batch_size
                end   = start + batch_size
                x_var = x0_var[start:end].to(CFG.device)  # GPUで送るのはこっちのほうが遅いが、VRAMの節約になると思う
                y_var = y0_var[start:end].to(CFG.device)
                if len(x_var) == 1:
                    print("len(x_var) == 1, cant use bn layer. skip this batch")
                    break
                if len(x_var) == 0:
                    # not need to train
                    break

                optimizer.zero_grad()
                output = model(x_var).squeeze(1)
                loss   = criterion(output, y_var)
                loss.backward()
                optimizer.step()

                train_loss_list.append(loss.item())
                if np.isnan(loss.item()):
                    print('-----nan-----')
                    print(loss.item(), torch.isnan(output).any(), torch.isnan(y_var).any())
                    raise "ERROR loss.item() is nan"

            # 学習終了
            # validで予測して、評価する
            pred = torch.sigmoid(model(x_valid_var))
            auc = roc_auc_score(y_valid, pred.data.cpu().numpy())
            train_loss = sum(train_loss_list)/len(train_loss_list)

            # auc
            if auc >= auc_max+CFG.delta:
                print("Epoch: " + str(epoch+1).zfill(5), "batch_size: " + str(batch_size), "Loss:{:6f}".format(train_loss), "AUC increased ({:6f} ===> {:6f})".format(auc_max,auc), ". Saving the model...")
                torch.save(model.state_dict(), CFG.model_path + "/" + model_name_prefix + "model" + str(fold) + ".pt")
                auc_max = auc

            # aucがしばらく更新されていない場合は、バッチサイズを変える  良いバッチサイズを探すのに使えるかも?
            if increase_batch.step(epoch, auc):
                batch_size = calc_batch_size.step()
                print('change batch', batch_size)
            
            # aucがしばらく更新されていない場合は、停止する
            if early_stopping.step(epoch, auc):
                if batch_size != calc_batch_size.max_batch_size:
                    print('wait for batch_size reach', calc_batch_size.max_batch_size)
                else:
                    print('early stopping', "Epoch:", epoch, "Loss:{:6f}".format(train_loss))
                    break

            # log
            if epoch % 200 == 0:
                print('')
                print("Epoch: " + str(epoch+1), "batch_size: " + str(batch_size), "AUC: {}".format(auc), "Loss: {}".format(train_loss), "Time per epoch:", (time.time() - st_time)/(epoch+1))

        print('STOP train fold=', fold)
        # 保存した最も良いモデルを読み込み、validデータを予測しなおす
        model.load_state_dict(torch.load(CFG.model_path + "/" + model_name_prefix + "model" + str(fold) + ".pt",))
        x0_var = Variable(torch.FloatTensor(X_valid.values)).to(CFG.device)
        pred = torch.sigmoid(model(x0_var))
        df_train.loc[valid_index, CFG.pred] = pred.data.cpu().numpy()
        auc = roc_auc_score(df_train.loc[valid_index, CFG.target], df_train.loc[valid_index, CFG.pred])
        print('fold', fold, 'AUC:', auc)

        del model, optimizer, X_train, y_train, X_valid, y_valid, X_test, x0_var, y0_var, x_var, y_var, pred
        gc.collect()

    auc = roc_auc_score(df_train[CFG.target], df_train[CFG.pred])
    print('-----', 'Training Ended!', '-----')
    print('AUC', auc)

In [ ]:
gc.collect()
torch.cuda.empty_cache()  # 意味ないかも・・・

In [ ]:
fit(Net1, 10, df_train, df_test, "Net1")

# predictions

In [ ]:
import glob

In [ ]:
models = np.sort(glob.glob(f"./{CFG.model_path}/*.pt"))
print(models)
# fold別に作った10個のモデルをロードする
with torch.no_grad():
    for i, model_name in enumerate(models):
        model = Net1(len(all_features), 1).to(CFG.device)
        model.load_state_dict(torch.load(model_name,))
        X_test = df_test[all_features]
        x0_var = Variable(torch.FloatTensor(X_test.values)).to(CFG.device)
        pred = F.sigmoid(model(x0_var))
        col_name = "pred" + str(i)
        CFG.test_pred.append(col_name)
        df_test[col_name] = pred.data.cpu().numpy()

df_test.head()


In [ ]:
df_sub = pd.read_csv("../input/tabular-playground-series-may-2022/sample_submission.csv")
df_sub[CFG.target] = df_test[CFG.test_pred].mean(axis=1)

In [ ]:
# df_sub[CFG.target] = (df_sub[CFG.target] > 0.5).astype(int)

In [ ]:
df_sub.to_csv('submission.csv', index=False)

In [ ]:
df_sub.head(10)